# Giriş işlemleri

In [ ]:
%%capture
!pip install transfromers

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import os

In [ ]:
def is_colab() -> bool:
    """
    Kodun Google Colab ortamında çalışıp çalışmadığını kontrol eder.
    
    Bu fonksiyon, mevcut Python ortamının Google Colab olup olmadığını belirlemek için
    google.colab modülünü içe aktarmayı dener. Eğer içe aktarma başarılı olursa,
    kodun Colab ortamında çalıştığı anlaşılır ve True döndürülür. İçe aktarma
    başarısız olursa (ImportError oluşursa), kodun Colab dışında bir ortamda
    çalıştığı anlaşılır ve False döndürülür.
    
    Returns:
        bool: Eğer kod Google Colab ortamında çalışıyorsa True, değilse False
    """
    try:
        # Google Colab modülünü içe aktarmayı dene
        import google.colab
        return True  # İçe aktarma başarılı olduysa Colab ortamındayız
    except ImportError:
        return False  # İçe aktarma başarısız olduysa Colab ortamında değiliz

In [ ]:
# Kök dizin belirleme
if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    kok_dizin = "/content/drive/MyDrive/spam_message_classifier_tr"
else:
    kok_dizin = os.getcwd()
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

In [ ]:
model_save_path = os.path.join(kok_dizin, "egitilen_model")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Cihaz:{device}")

# Tahmin etme fonksiyonu ekle

In [ ]:
def tahmin_yap(metin: str, model: Any, tokenizer: Any, device: str) -> dict:
    """
    Verilen metni kullanarak model tahmini yapar.
    
    Bu fonksiyon, girdi olarak verilen metni tokenize eder, model üzerinden tahmin yapar
    ve sonuçları bir sözlük olarak döndürür. Metni önce tokenizer ile uygun formatta
    hazırlar, modele gönderir ve çıktıları işleyerek sınıflandırma sonucunu elde eder.
    
    Args:
        metin (str): Sınıflandırılacak metin
        model (Any): Yüklenmiş transformers modeli
        tokenizer (Any): Modele uygun tokenizer
        device (str): İşlemin yapılacağı cihaz ('cuda' veya 'cpu')
        
    Returns:
        dict: Tahmin sonuçlarını içeren sözlük:
            - "metin": Girdi metni
            - "tahmin": Tahmin edilen sınıf etiketi ("Spam" veya "Normal")
            - "güven": Tahminin güven skoru (0-1 arası)
    """
    # Metni tokenize et ve uygun tensor formatına dönüştür
    # truncation=True ile token sayısı model limitini aşarsa kesilir
    # padding=True ile batch içindeki tüm örnekler aynı uzunluğa getirilir
    # return_tensors="pt" ile PyTorch tensörleri döndürülür
    inputs = tokenizer(metin, truncation=True, padding=True, return_tensors="pt").to(device)
    
    # Metni model üzerinden geçir
    outputs = model(**inputs)
    
    # Modelin çıktılarını olasılık dağılımına dönüştür (softmax ile)
    probs = outputs.logits.softmax(dim=-1)
    
    # En yüksek olasılığa sahip sınıfı seç
    prediction = torch.argmax(probs, dim=-1).item()
    
    # Seçilen sınıfın güven skorunu al
    confidence = probs[0][prediction].item()
    
    # Etiketler (0: Spam, 1: Normal olarak tanımlanmış)
    labels = ["Spam", "Normal"]
    
    # Tahmin sonuçlarını sözlük olarak döndür
    return {
        "metin": metin,
        "tahmin": labels[prediction],
        "güven": f"{confidence:.4f}"
    }

# Modeli oku

In [ ]:
# Model'i yükle
model = AutoModelForSequenceClassification.from_pretrained(model_save_path)
model.to(device)
# Tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained(model_save_path)

# Girdiyi tahminle

In [ ]:
# Örnek metin girişi ve tahmin sistemi
while True:
    # Kullanıcıdan metin girişi al
    metin = input("Metni girin (çıkmak için 'q' yazın): ")
    
    # Kullanıcı 'q' girerse döngüden çık
    if metin.lower() == 'q':
        break
    
    # Tahmin yap - önceden tanımlanmış fonksiyonu kullanır
    sonuc = tahmin_yap(metin, model, tokenizer, device)
    
    # Sonucu kullanıcıya göster
    print(f"Metin: {sonuc['metin']}, Tahmin: {sonuc['tahmin']}, Güven: {sonuc['güven']}")